### **Approach 1: Creating and populating the database with SQLite**

Since SQLite is server-less and self-contained I figured this would be a good starting point for a prototype.\
I created some scripts with sqlite3 to create and populate the database with authors, books, stores, customers, orders etc.

I ended up with these tables:

<div style="display: flex; flex-direction: row;">
  <div style="flex: 1; margin:5px;">

**Table 1: Author**
- 'ID' (Primary Key)
- 'Name'
- 'Surname'
- 'Birthdate'

**Table 2: Book**
- 'ISBN13' (Primary Key)
- 'Title'
- 'Language'
- 'Price'
- 'Release'
- 'AuthID'


**Table 3: Store**
- 'ID' (Primary Key)
- 'Store_Name'
- 'Store_Address'

</div>
<div style="flex: 1; margin:5px;">

**Table 4: Inventory**
- 'StoreID'
- 'ISBN13'
- 'Stock'
- Composite Key: 'StoreID' & 'ISBN13'

**Table 5: Cst (Customer)**
- 'ID' (Primary Key)
- 'Name'
- 'Surname'
- 'Address'
- 'City'
- 'State'
- 'ZipCode'
- 'Email'

</div>
<div style="flex: 1; margin:5px;">

**Table 6: Cst_Order**
- 'OrderID' (Primary Key)
- 'CustomerID'
- 'Order_Date'
- 'Expected_Delivery'
- 'Total_Price'

**Table 7: Cst_Order_Items**
- 'OrderID (Primary Key)
- 'ISBN13' (Primary key)
- 'Units'
- 'Shipped_From'
- Composite Key: 'OrderID' & 'ISBN13'
</div>

The requriement of the view *Vy: ”TitlarPerFörfattare”*, was created with some joins:

In [ ]:
c.execute('''CREATE VIEW TitlarPerFörfattare AS
               SELECT Author.Name || ' ' || Author.Surname AS Namn,
                      strftime('%Y', 'now') - strftime('%Y', Author.Birthdate) AS Ålder,
                      COUNT(DISTINCT Book.Title) AS Titlar,
                      SUM(Book.Price * Inventory.Stock) AS Lagervärde
               FROM Author
               JOIN Book ON Author.ID = Book.AuthID
               JOIN Inventory ON Book.ISBN13 = Inventory.ISBN13
               GROUP BY Author.Name, Author.Surname
               ORDER BY Namn''')

**Sample data**, such as customer info was easy to generate with chatGPT.\
I made a function called populate_customer_table(), in which I added these rows:

In [ ]:
    customer_data = [
        ('John', 'Doe', 'johndoe@example.com', '123 Main St', 'New York', 'NY', '10001'),
        ('Jane', 'Smith', 'janesmith@example.com', '456 Elm St', 'Los Angeles', 'CA', '90001'),
        ('Bob', 'Johnson', 'bobjohnson@example.com', '789 Oak St', 'Chicago', 'IL', '60601'),
        ('Alice', 'Lee', 'alicelee@example.com', '321 Maple St', 'Houston', 'TX', '77001'),
        ('David', 'Brown', 'davidbrown@example.com', '654 Pine St', 'Philadelphia', 'PA', '19101')
    ]

But I soon realized I'd actually already made a program for this, <a href="https://github.com/wlinds/tpme">tpme</a>.

tpme can generate names and email addresses but **not** physical home addresses.

So should I focus on adding support for home address generation in tpme or should I port the SQLite code to a larger dbms?

Neither -- *I have like 120 hours left until deadline. No problem. I'll work on some front end stuff.*

**Front-end hours**

I started working on an interface with Flask to allow users to purchase books. But then it struck me that this was a total waste of time, I should rather make an interface for the shopkeepers at the store. Then we just outsource the customer-site creation and just hand them some API.

Something that would be useful for both management and customers would be a search function. 

**Search function**

In [2]:
from books import search_books

search_books('Hitchhiker')

{"The Hitchhiker's Guide to the Galaxy": [('123 Main St.', 86),
  ('456 Elm St.', 32),
  ('789 Oak St.', 30)]}

### **Writing and burning books**

I created a function to add books and to remove (burn) books. Adding books was easy (except for ISBN checksum I which I messed up), will return to this later. But the burn_book() function caused some errors.

The burn_book() worked flawelessly for any newly added book, but if I tried to burn books that were currently in stock I received the error:

<code>
AssertionError: Dependency rule tried to blank-out primary key column 'Inventory.ISBN13' on instance
</code>

I think the foreign key constraint was trying to blank-out the primary key column ISBN13 on the instance of Inventory instead of deleting the row referencing the book. Of course, this is not allowed because the ISBN13 column is part of the primary key of the Inventory table, and therefore cannot be null or empty.

To solve this, I had to update the logic in the burn_book function to check for and delete any referencing rows in the Inventory table before attempting to burn the book from the Book table.

### **Stocking books (add to Inventory table)**

After adding books to the table Book, we might wanna add these books to our Inventory/Stock. This again caused an error if we already had the book in stock. To solve this we first make a query to check if the book is in stock or not.

In [ ]:
def add_to_inventory(isbn, store_id, stock):
    session = Session()
    inventory = session.query(Inventory).filter_by(ISBN13=isbn, StoreID=store_id).first()
    if inventory:
        inventory.Stock += stock
    else:
        inventory = Inventory(ISBN13=isbn, StoreID=store_id, Stock=stock)
        session.add(inventory)
    session.commit()

Now we have functions to search for existing books, write new books, burn books and stock books. These functions should be accessed by the bookstore staff. But what about our clients, customers, resellers, etc. They should be able to order books!

### **Actually selling books (or buying, depending on who you are)**
I made a new function which allows any registered customer to purchase any book if its in stock.

In [1]:
from customer import purchase_book

purchase_book("9780007117116", 1, 1, 2)

John purchased 2 copies of 'The Lord of the Rings' (ISBN: 9780007117116) from '123 Main St.' for a total cost of 59.98.
There are 4 copies of 'The Lord of the Rings' (ISBN: 9780007117116) left in stock at '123 Main St.'.


In [3]:
purchase_book("9780007117116", 1, 1, 9999)

Sry, out of stock in this store. Try another store.


Some basic error handling.

To make this work as I wanted I also had to create a new table:

# Transactions

Transactions contain ISBN, StoreID, CustomerID, Quantity, DateTime of Purchase and Total_Cost.

Its like 80 hours left until deadline and I have so many ideas. But I really should work on improving the base code in models.py before adding more stuff. I've added a TODO:

In [ ]:
#TODO:
# Update to strict naming conventions (snake_case)
# Reduce redundancy (DRY)
# Improve constraints
# Remove declarative_base() (Legacy code)

I will probably let that TODO sit there for a while, maybe indefinitely.

Ok, so I've actually updated mostly all names to snake_case! 🐍 While at it I made the column names a bit more intuitive. I have some names left, StoreID, for example. But I just have to go through all the functions in store_manager.py to adjust this.

### **Moving books**

In [1]:
from store_manager import *

move_books(9780007117116, 2, 3, 1000)

1000 copies of ISBN 9780007117116 moved from store 2 to store 3.


True

## **Testing**

Running this in ipynb is weird but let's try it. Testing with new instance from scratch:

In [6]:
from models import *
import os, store_manager, books, customer
from Scripts.utils import titles_by_author, get_title, total_sales
def some_testing():

    Base.metadata.create_all(bind=engine)

    store_manager.get_dummy_stores()
    books.get_dummy_books()

    books.get_dummy_authors()

    # Add 200 copies of Lord of The Rings to Store 1
    books.add_to_inventory(9780007117116, 1, 200, verbose=True)

    # Add 5000 copies of ALL existing books to store 2:
    books.add_all_books(store_id=2, copies=5000, verbose=True)

    # Add view #TODO: Missing date of birth
    titles_by_author()

    # Add dummy customers
    customer.get_dummy_cst()

    # Search book
    results = books.search_books("The")
    for i in results:
        print(i)

    # Customer book purchase
    customer.purchase_book(9780007117116, 1, 1, 10)
    customer.purchase_book(9780007117116, 2, 2, 1)
    # Move books
    store_manager.move_books(9780007117116, 2, 3, 1000)
    store_manager.move_books(9780007117116, 1, 2, 190)
    customer.purchase_book("9780007117116", 1, 3, 3)
    customer.purchase_book("9780007117116", 3, 3, 3)
    total_sales()

some_testing()

Store 'Toads Books' already exists
Store 'The Great Library' already exists
Store 'Böcker & Babbel' already exists
Book '9780007117116' already exists as The Lord of the Rings.
Book '9780439554930' already exists as Harry Potter and the Philosopher's Stone.
Book '9780553801477' already exists as Foundation.
Book '9780553588488' already exists as Ender's Game.
Book '9781400031702' already exists as The Picture of Dorian Gray.
Book '9780141187761' already exists as Nineteen Eighty-Four.
Book '9780316346627' already exists as A Game of Thrones.
Book '9780061124952' already exists as American Gods.
Book '9780679745587' already exists as One Hundred Years of Solitude.
Book '9780553382563' already exists as The Hitchhiker's Guide to the Galaxy.
Book '9781473214712' already exists as Good Omens.
Added 200 copies of 9780007117116 to Toads Books.
Added 5000 of all existing books to The Great Library
View already exists.
The Lord of the Rings
Harry Potter and the Philosopher's Stone
The Picture 

And we run it again to check error handling:

Issue occurs when Jane tries to make a purchase:
```customer.purchase_book(9780007117116, 2, 2, 1)```\
(isbn, store_id, customer_id, quantity)

In the function we call ```get_customer(customer_id)``` which runs a query to simply print the name of the customer_id:

```Session().query(Customer.name).filter_by(ID=customer_id).scalar()```

Using .scalar() here is not necessary since we only every use this to query for 1 and only 1 name. I'll change that.

Ok new error - This one's either caused by leftover code. ```session.commit()``` is not needed when we run ```Session()``` context manager in SQLAlchemy. Or it could be that the db is open in another Jupyter notebook kernel? Let's try remove commit(), restart and try again:

EDIT: I was wrong. ```session.commit()``` commit() WAS needed. I guess the issue was within Jupyter notebook kernel.

Nice! I don't like Jupyter Notebook and SQL at all though. It connects to another instance of the db, which is fine for now, but in the long run I gotta find another workflow.

### **Multiple authors**
We need to create a many-to-many relationship for multiple authors. But I'm not entirely sure what the best design would be. I will get back to this. TODO

Uhhh lets check what else:

| Requirement                                                    | Status       |
| -------------------------------------------------------------- | ------------ |
| Minst 6 (4+2) tabeller relevanta för bokhandeln. | s'all good i got 7 as of now |
| Tabeller ska vara normaliserade enligt 3NF.                    | yeah kinda i guess |
| Alla relationer ska ha PK och FK nycklar, samt relationsvilkor som förhindrar FK's att peka på nycklar som inte existerar. | yeah kinda |
| Kolumner ska använda lämpliga datatyper för den typ av information som ska lagras. | yeah kinda... |
| Databasen ska innehålla ett ER-diagram som visar relationerna mellan alla tabeller på ett tydligt sätt. | nope |
| Databasen ska innehålla lämpliga testdata för demonstration.   | yes          |
| Databasen ska innehålla vyn ”TitlarPerFörfattare” enligt specifikation. | yes          |
| Redovisa pythonkod, med funktionalitet enligt beskrivning ovan. | yes          |




extras

Requirement | Status
--- | ---
Databasen ska innehålla minst 8 (4+4) tabeller relevanta för bokhandeln. (Det ska vara minst 8 entiteter. Junktion tables räknas ej.) | Nah, we still 7
Databasen är gjord så böcker kan ha flera författare. | nah, working on it tho
Databasen ska innehålla SP ”FlyttaBok” enligt specifikation. | i mean kinda
Databasen har ytterligare en vy som sammanställer för bokhandeln relevant information från minst två av de tabeller som du lagt till under rubriken ”Övriga tabeller”. Motivera hur bokhandeln kan ha nytta av den sammanställda vyn. | not yet


---

**About ISBN**

Example of how an ISBN is structured: ISBN 978-91-7000-150-5

1. Prefix - all ISBNs begin with the prefix 978
2. Area identifier - for nationally, geographically or linguistically determined areas (91 = Sweden)
3. Publisher identifier (7000)
4. Title identifier (150)
5. Check digit (5)

<a href="https://www.kb.se/download/18.280a529018702e6727e363d/1681904094677/ISBN-2023.pdf">source</a>

This might cause search issues. Users might query a seach as "978-91-7000-150-5", "978-9170001505" or "9789170001505".

To prevent this, we run <code>replace('-','').replace(' ', '')</code> which removes all dashes and blankspaces.

In [3]:
from Scripts.utils import validate_isbn

# Validate ISBN
# Source https://rosettacode.org/wiki/ISBN13_check_digit

# Validate the check digit of an ISBN-13 code:

#  Multiply every other digit by  3.
#  Add these numbers and the other digits.
#  Take the remainder of this number after division by  10.
#  If it is  0,   the ISBN-13 check digit is correct.

tests = '''
978-91-7000-150-5
978-1734314509
978-1788399081
978-1788399083'''.strip().split()
for t in tests:
    print(f"ISBN13 {t} validates {validate_isbn(t)}")

ISBN13 978-91-7000-150-5 validates True
ISBN13 978-1734314509 validates False
ISBN13 978-1788399081 validates True
ISBN13 978-1788399083 validates False


Also, ISBN should be stored as text data and not int, since some ISBN can start with 0.